In [1]:
import pandas as pd
import regex as re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt')
pd.set_option('display.max_rows',None)
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Data MUA for Memory Based System Recommendation

In [2]:
class ContentBasedFiltering:
    def case_folding(self, text):
        if type(text) == str:
            text = text.lower()
        return text

    def punctuation_removal(self, text):
        if type(text) == str:
            text = re.sub(r'[^\w\s]', '', text)
        return text

    def tokenize(self, text):
        tokens = word_tokenize(text)
        return tokens
        
    def stopwords_removal(self, text):
        stopword_factory = StopWordRemoverFactory()
        stopword_remover = stopword_factory.create_stop_word_remover()
        no_stopword = stopword_remover.remove(text)
        return no_stopword

    def preprocessing(self, text):
        text = self.case_folding(text)
        text = self.punctuation_removal(text)
        text = self.stopwords_removal(text)
        return text

In [3]:
data = pd.read_excel('./web_mua/static/Data MUA.xlsx')
data.head()

,no,nama,nama_MUA,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude,longtitude,rating
0,1,Anggia Pradita A,Intan Rias Pengantin,la tulipe cover foundation,natural (putih),light,cool,Murah,-8.233613,113.66733,4
1,1,Anggia Pradita A,Intan Rias Pengantin,la tulipe cover foundation,suntan (sawo matang),medium,warm,Murah,-8.233613,113.66733,4
2,2,VIKA APRILIA SINTA PUTRI,Sanggar Rias Cantik Tina,la tulipe cover foundation,rachel 1 (kuning langsat),medium,neutral,Murah,-8.233613,113.66733,5
3,2,VIKA APRILIA SINTA PUTRI,Sanggar Rias Cantik Tina,la tulipe cover foundation,suntan (sawo matang),medium,warm,Murah,-8.233613,113.66733,5
4,3,septia maharani,Rias Pengantin Azizah,la tulipe cover foundation,natural (putih),light,cool,Sedang,-8.233613,113.66733,3


In [4]:
def cleaning_data(data):
    cleaned_data = []
    CBF = ContentBasedFiltering()
    for i in range(len(data)):
        cleaned_data.append(CBF.preprocessing(str(data[i])))
    return cleaned_data

In [5]:
#praproses data untuk nama
data['nama'] = cleaning_data(data['nama'])

In [6]:
data_for_model = data.copy()
data_for_user = data.copy()

#praproses data untuk deskripsi
data_for_model['desc'] = data['produk_makeup'] + ' ' + data['shade'] + ' ' + data['skin_color'] + ' ' + data['skin_undertone']
data_for_model['desc'] = cleaning_data(data_for_model['desc'])
data_for_user['desc'] = data['produk_makeup'] + ' ' + data['skin_color'] + ' ' + data['skin_undertone']
data_for_user['desc'] = cleaning_data(data_for_user['desc'])

In [7]:
# data_for_model = data_for_model.groupby('no').agg({'nama':'first', 'nama_MUA':'first', 'kategori_harga': 'first', 'latitude':'first', 'longtitude':'first', 'rating':'first','desc':' '.join}).reset_index()
data_for_model = data_for_model.sort_values(by=['nama'], ignore_index=True)

data_for_user = data_for_user.sort_values(by=['nama'], ignore_index=True)
data_for_user = data_for_user.drop(columns=['shade'])

In [8]:
data_for_model.head()

,no,nama,nama_MUA,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,Sedang,-8.193711,113.616830,5,lt pro smooth corrector cream foundation natur...
1,79,adera zia,Bonita MUA Jember,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,Sedang,-8.139019,113.728548,4,lt pro smooth corrector cream foundation natur...
2,79,adera zia,Bonita MUA Jember,la tulipe cover foundation,suntan (sawo matang),medium,warm,Sedang,-8.139019,113.728548,4,la tulipe cover foundation suntan sawo matang ...
3,48,adera zia,Ardie Make Up & Salon,la tulipe cover foundation,suntan (sawo matang),medium,warm,Sedang,-8.157255,113.652074,3,la tulipe cover foundation suntan sawo matang ...
4,48,adera zia,Ardie Make Up & Salon,LT Pro smooth corrector cream foundation,plum rose (putih),light,cool,Sedang,-8.157255,113.652074,3,lt pro smooth corrector cream foundation plum ...


In [9]:
data_for_user.head()

,no,nama,nama_MUA,produk_makeup,skin_color,skin_undertone,kategori_harga,latitude,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,LT Pro smooth corrector cream foundation,medium,neutral,Sedang,-8.193711,113.616830,5,lt pro smooth corrector cream foundation mediu...
1,79,adera zia,Bonita MUA Jember,LT Pro smooth corrector cream foundation,medium,neutral,Sedang,-8.139019,113.728548,4,lt pro smooth corrector cream foundation mediu...
2,79,adera zia,Bonita MUA Jember,la tulipe cover foundation,medium,warm,Sedang,-8.139019,113.728548,4,la tulipe cover foundation medium warm
3,48,adera zia,Ardie Make Up & Salon,la tulipe cover foundation,medium,warm,Sedang,-8.157255,113.652074,3,la tulipe cover foundation medium warm
4,48,adera zia,Ardie Make Up & Salon,LT Pro smooth corrector cream foundation,light,cool,Sedang,-8.157255,113.652074,3,lt pro smooth corrector cream foundation light...


In [10]:
data_for_model.shape

(179, 12)

In [11]:
from sklearn.preprocessing import LabelEncoder

harga_labels = LabelEncoder()
harga_labels = harga_labels.fit(data['kategori_harga'])
harga_labels.classes_ = np.array(['Murah', 'Sedang', 'Mahal'])
data_for_model['kategori_harga'] = harga_labels.transform(data_for_model['kategori_harga'])
data_for_user['kategori_harga'] = harga_labels.transform(data_for_user['kategori_harga'])

In [12]:
data_for_model.head()

,no,nama,nama_MUA,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,1,-8.193711,113.616830,5,lt pro smooth corrector cream foundation natur...
1,79,adera zia,Bonita MUA Jember,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,1,-8.139019,113.728548,4,lt pro smooth corrector cream foundation natur...
2,79,adera zia,Bonita MUA Jember,la tulipe cover foundation,suntan (sawo matang),medium,warm,1,-8.139019,113.728548,4,la tulipe cover foundation suntan sawo matang ...
3,48,adera zia,Ardie Make Up & Salon,la tulipe cover foundation,suntan (sawo matang),medium,warm,1,-8.157255,113.652074,3,la tulipe cover foundation suntan sawo matang ...
4,48,adera zia,Ardie Make Up & Salon,LT Pro smooth corrector cream foundation,plum rose (putih),light,cool,1,-8.157255,113.652074,3,lt pro smooth corrector cream foundation plum ...


In [13]:
data_for_user.head()

,no,nama,nama_MUA,produk_makeup,skin_color,skin_undertone,kategori_harga,latitude,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,LT Pro smooth corrector cream foundation,medium,neutral,1,-8.193711,113.616830,5,lt pro smooth corrector cream foundation mediu...
1,79,adera zia,Bonita MUA Jember,LT Pro smooth corrector cream foundation,medium,neutral,1,-8.139019,113.728548,4,lt pro smooth corrector cream foundation mediu...
2,79,adera zia,Bonita MUA Jember,la tulipe cover foundation,medium,warm,1,-8.139019,113.728548,4,la tulipe cover foundation medium warm
3,48,adera zia,Ardie Make Up & Salon,la tulipe cover foundation,medium,warm,1,-8.157255,113.652074,3,la tulipe cover foundation medium warm
4,48,adera zia,Ardie Make Up & Salon,LT Pro smooth corrector cream foundation,light,cool,1,-8.157255,113.652074,3,lt pro smooth corrector cream foundation light...


## User Input

In [14]:
# use this parameter to get some MUA recommendations
def data_preparation(data_for_model, data_for_user, index_user, user):
    du = data_for_user.copy()
    query_data = du.iloc[[index_user]].reset_index()
    query_data_todict = query_data.to_dict('index')[0]
    # part_of_model = dm[dm.index != user].reset_index()
    part_of_model = data_for_model.copy()
    part_of_query = part_of_model.loc[part_of_model['nama'] == user].reset_index()
    part_of_model.loc[part_of_model['nama'] == user, 'rating'] = 0
    part_of_model = part_of_model.reset_index()
    part_of_model = part_of_model.drop(columns='index')
    return query_data_todict, part_of_query, part_of_model

# example of input data
find_user = data_for_user.index[data_for_user['nama'] == 'adera zia'].to_list()
txt_user = 'adera zia'
query_data_todict, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, find_user[0], txt_user)
# part_of_model = data_preparation(data_for_model, data_for_user, find_user[0], txt_user, first=False)

## Calculating Cosine Similarity for Content Based Model

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity

def tf_idf_vector(part_of_model, query_data_todict):
    copy_md = part_of_model.copy()
    md = pd.DataFrame(copy_md["desc"])
    md.loc[len(md)] = query_data_todict["desc"]
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(md["desc"])
    model = X[:len(part_of_model)]
    query = X[len(part_of_model):]
    return model, query

In [16]:
from geopy.distance import great_circle
import math
def calculate_distance(query_data_todict, part_of_model):
        loc1 = (query_data_todict['latitude'], query_data_todict['longtitude'])
        distance_loc = []
        for i in range(len(part_of_model.index)):
            loc2 = (part_of_model['latitude'][i], part_of_model['longtitude'][i])
            tmp = great_circle(loc1, loc2).km
            value = math.ceil(tmp * 100) / 100
            distance_loc.append(value)
        return distance_loc

In [17]:
def get_numtext_content(query_data_todict, part_of_model):
    tfidf_model, tfidf_query = tf_idf_vector(part_of_model, query_data_todict)
    cos_sim = cosine_similarity(tfidf_query, tfidf_model)
    data_sim = cos_sim.reshape(-1,1)
    distance = calculate_distance(query_data_todict, part_of_model)
    return data_sim, distance

In [18]:
from sklearn.preprocessing import StandardScaler

def get_values(part_of_model):
    tmp_data = part_of_model[['kategori_harga','sim_final']].to_numpy()
    sim_num = cosine_similarity(tmp_data)
    sim_num = np.array([np.mean(p) for p in sim_num])
    for i in range(sim_num):
        if sim_num < 0.0:
            sim_num[i] = 0.0
    return sim_num

In [49]:
def value_of_mua(query_data_todict, part_of_model):
    sim, distance = get_numtext_content(query_data_todict, part_of_model)
    part_of_model["sim"] = sim
    part_of_model["distance"] = distance
    sf = []
    for i in range(len(part_of_model)):
        sf.append(part_of_model["sim"][i]/(part_of_model["distance"][i] + 1))
    part_of_model["sim_final"] = sf   
    sim_num = get_values(part_of_model)
    part_of_model["sim_num"] = sim_num
    part_of_model = part_of_model.sort_values(by=['nama_MUA'])
    nama_mua = part_of_model["nama_MUA"].unique()
    value_mua = np.empty(len(nama_mua), dtype=float)
    for i in range(len(nama_mua)):
        tmp_val = []
        for j in range(len(part_of_model.index)):
            if nama_mua[i] == part_of_model.nama_MUA[j]:
                tmp_val.append(part_of_model["sim_num"][j])
        value_mua[i] = np.mean(tmp_val)
    part_of_model = part_of_model.reset_index()
    part_of_model = part_of_model.drop(columns='index')
    return nama_mua, value_mua, part_of_model

## FIND USER

In [50]:
unique_user = data_for_model['nama'].unique()
unique_user
query_user = ['adera zia', 'anggia pradita a', 'anissa serina', 'calista devi aqilah', 'callista febriana erwanto', 'dwi rahma', 'maulina vira', 
              'vika aprilia sinta putri', 'yuricha mahargiyani wijaya', 'zainata hanum masruroh', 'zyanti laila']
selected_user = []
for i in query_user:
    find_user = data_for_model.index[data_for_model['nama'] == i].to_list()
    selected_user.append(find_user[0])
selected_user

[0, 10, 20, 33, 45, 73, 101, 142, 151, 161, 169]

## HYBRID FILTERING

In [51]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing as pre
import statistics 

In [105]:
#chosen user for testing
def hybrid_evaluation(data_for_model, data_for_user, selected_user, query_user):
    mae_hy = []
    for u in range(len(selected_user)):
        query_data_todict, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[u], query_user[u])
        nama_mua, value_mua, part_of_model = value_of_mua(query_data_todict, part_of_model)
        pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        col_model = pivot.columns.tolist()
        val_model = np.array(pivot.values.tolist(), dtype=float)
        nama_user = np.array(pivot.index.tolist())
        pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_query = pivot_query.values.tolist()
        col_query = pivot_query.columns.tolist()
        feature = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_query)):
                if col_model[i] == col_query[j]:
                    feature[i] = val_query[0][j]
                    break
        cos_sim_user = cosine_similarity([feature], val_model)

        weight_user = np.empty(len(part_of_model), dtype=float)
        for i in range(len(part_of_model)):
            for j in range(len(nama_user)):
                if part_of_model["nama"][i] == nama_user[j]:
                    weight_user[i] = cos_sim_user[0][j]
        
        weight_content = np.empty(len(part_of_model), dtype=float)
        for i in range(len(part_of_model)):
            for j in range(len(nama_mua)):
                if part_of_model["nama_MUA"][i] == nama_mua[j]:
                    weight_content[i] = value_mua[j]
        
        final_res = part_of_model.copy()
        final_res = final_res[['nama', 'nama_MUA']]
        final_res['user'] = weight_user
        final_res['content'] = weight_content
        
        mean_weight = np.empty(len(part_of_model), dtype=float)
        for i in range(len(part_of_model)):
            mean_weight[i] = statistics.mean([weight_user[i],  weight_content[i]]) 
        final_res['mean_'] = mean_weight
        final_res = final_res.sort_values(by=['mean_'], ignore_index=True, ascending=False)
        get_user = final_res.groupby("nama").mean_.agg(["mean"])
        list_nama = get_user.index[:10].tolist()
        pivot = part_of_model.copy()
        pivot['mean_'] = mean_weight
        pivot = pivot[pivot.nama.isin(list_nama)].reset_index()
        pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
        pivot_['weight'] = pivot.groupby("nama_MUA").mean_.agg(['mean'])
        value_rating = np.zeros(len(col_model), dtype=float)
        for i in range(len(pivot_.index)):
            for j in range(len(col_model)):
                if value_rating[j] == 0.0:
                    if col_model[j] == pivot_.index[i]:
                        if pivot_['weight'][i] != 0.0:
                            value_rating[j] = pivot_["mean"][i] * pivot_['weight'][i]
                        else:
                            value_rating[j] = weight_content[i]
                else:
                    continue
     
        tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
        mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
        pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_label = pivot_label.values.tolist()
        col_label = pivot_label.columns.tolist()
        final_label = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_label)):
                if col_model[i] == col_label[j]:
                    final_label[i] = val_label[0][j]
                    break
        mae = mean_absolute_error(value_rating, final_label)
        mae_hy.append(mae)
        
    mean_mae_hy = np.mean(mae_hy)
    return mua_recs, mean_mae_hy, mae_hy

In [106]:
hybrid_based_fil, mae_hyb, mae_eval = hybrid_evaluation(data_for_model, data_for_user, selected_user, query_user)

In [107]:
hybrid_based_fil[hybrid_based_fil.score > 0.0]

,nama_mua,score
0,Dinda Wijaya Make Up,2.764755
1,Laura Make Up & Bridal,2.316865
2,Bonita MUA Jember,2.016189
3,Ulfa Beauty MUA,1.942083
4,Larasnada Make Up,1.751060
5,Liyan Make Up Jember,1.682219
6,Abha Beauty Make Up Artist,1.581644
7,Rias Manten Mariyam,1.565666
8,Sanggar Rias Nining,1.557969
9,Khaliza Make Up,1.539427


In [108]:
mae_hyb

0.7729057643146419

## TESTING

In [116]:
def testing_user(data_for_model, data_for_user, index_usr):
    query_data_todict, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[index_usr], query_user[index_usr])
    nama_mua, value_mua, part_of_model = value_of_mua(query_data_todict, part_of_model)
    pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
    col_model = pivot.columns.tolist()
    val_model = np.array(pivot.values.tolist(), dtype=float)
    nama_user = np.array(pivot.index.tolist())
    pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
    val_query = pivot_query.values.tolist()
    col_query = pivot_query.columns.tolist()
    feature = np.zeros(len(col_model), dtype=float)
    for i in range(len(col_model)):
        for j in range(len(col_query)):
            if col_model[i] == col_query[j]:
                feature[i] = val_query[0][j]
                break
    cos_sim_user = cosine_similarity([feature], val_model)

    weight_user = np.empty(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        for j in range(len(nama_user)):
            if part_of_model["nama"][i] == nama_user[j]:
                weight_user[i] = cos_sim_user[0][j]
        
    weight_content = np.empty(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        for j in range(len(nama_mua)):
            if part_of_model["nama_MUA"][i] == nama_mua[j]:
                weight_content[i] = value_mua[j]
        
    final_res = part_of_model.copy()
    final_res = final_res[['nama', 'nama_MUA']]
    final_res['user'] = weight_user
    final_res['content'] = weight_content

    mean_weight = np.zeros(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        mean_weight[i] = statistics.harmonic_mean([weight_user[i],  weight_content[i]]) 
    final_res['mean_'] = mean_weight
    final_res = final_res.sort_values(by=['mean_'], ignore_index=True, ascending=False)
    get_user = final_res.groupby("nama").mean_.agg(["mean"])
    list_nama = get_user.index[:10].tolist()
    pivot = part_of_model.copy()
    pivot['mean_'] = mean_weight
    pivot = pivot[pivot.nama.isin(list_nama)].reset_index()
    distance = pivot[['nama_MUA','distance']]
    produk = pivot[['nama_MUA', 'sim']]
    final_ = pivot[['nama_MUA', 'sim_final']]
    desc = pivot[['nama_MUA', 'produk_makeup', 'shade', 'skin_color', 'skin_undertone']]
    desc['desc'] = desc['produk_makeup'] + ' - ' + desc['shade'] + ' - ' + desc['skin_color'] + ' - ' + desc['skin_undertone']
    pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
    pivot_['nama_MUA'] = desc.groupby('nama_MUA').first().index.to_list()
    pivot_['weight'] = pivot.groupby("nama_MUA").mean_.agg(['mean'])
    pivot_['score'] = pivot_["mean"] * pivot_['weight']
    pivot_['desc'] = desc.groupby('nama_MUA').agg({'desc': lambda x: list(x)}).reset_index()['desc'].values
    pivot_['distance'] = distance.groupby("nama_MUA").distance.agg(['mean'])
    pivot_['sim'] = produk.groupby("nama_MUA").sim.agg(['mean'])
    pivot_['sim_final'] = final_.groupby("nama_MUA").sim_final.agg(['mean'])
    pivot_ = pivot_.sort_values(by=['score'], ignore_index=True, ascending=False)
    return pivot_

In [117]:
#highest MAE
mae_eval[mae_eval.index(min(mae_eval))]

0.6679172598842177

In [118]:
#highest
idx_high = mae_eval.index(min(mae_eval))
data_for_model.iloc[[selected_user[idx_high]]]

,no,nama,nama_MUA,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude,longtitude,rating,desc
151,23,yuricha mahargiyani wijaya,Ayunia Bridal dan Salon,Wardah Lightening Liquid Foundation 25ml,neutral beige,light,warm,0,-8.17887,113.696615,5,wardah lightening liquid foundation 25ml neutr...


In [119]:
h_pivot = testing_user(data_for_model, data_for_user, idx_high)
h_pivot[h_pivot.score > 0.0]

,count,mean,nama_MUA,weight,score,desc,distance,sim,sim_final
0,3,5.000000,Rias Manten Mariyam,0.351303,1.756513,[la tulipe cover foundation - natural (putih) ...,5.68,0.050239,0.007521
1,3,5.000000,Eka Make Over,0.350697,1.753484,[LT Pro smooth corrector cream foundation - na...,61.87,0.069244,0.001101
2,1,5.000000,Laura Make Up & Bridal,0.348166,1.740829,[la tulipe cover foundation - natural (putih) ...,2.22,0.112136,0.034825
3,2,5.000000,Dinda Wijaya Make Up,0.331278,1.656392,[LT Pro smooth corrector cream foundation - na...,8.94,0.053925,0.005425
4,1,4.000000,Ulfa Beauty MUA,0.361348,1.445392,[la tulipe cover foundation - suntan (sawo mat...,48.48,0.090221,0.001823
5,1,4.000000,Liyan Make Up Jember,0.322534,1.290138,[la tulipe cover foundation - natural (putih) ...,1075.75,0.112136,0.000104
6,2,4.000000,Bonita MUA Jember,0.316128,1.264512,[la tulipe cover foundation - suntan (sawo mat...,5.66,0.053925,0.008097
7,3,3.666667,Larasnada Make Up,0.321882,1.180232,[la tulipe cover foundation - suntan (sawo mat...,8.35,0.073329,0.007843
8,4,3.500000,Ardie Make Up & Salon,0.322218,1.127764,[la tulipe cover foundation - natural (putih) ...,5.46,0.087713,0.013578
9,3,3.000000,Rias Pengantin Criepno Lestari,0.350771,1.052312,[LT Pro smooth corrector cream foundation - ye...,28.19,0.072567,0.002486


In [120]:
#lowest MAE
mae_eval[mae_eval.index(max(mae_eval))]

0.8419527904247506

In [121]:
#lowest
idx_low = mae_eval.index(max(mae_eval))
data_for_model.iloc[[selected_user[idx_low]]]

,no,nama,nama_MUA,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude,longtitude,rating,desc
142,74,vika aprilia sinta putri,Sanggar Rias Pengantin Dewi Balqis,la tulipe cover foundation,natural (putih),light,cool,0,-8.163596,113.703364,3,la tulipe cover foundation natural putih light...


In [122]:
h_pivot = testing_user(data_for_model, data_for_user, idx_low)
h_pivot[h_pivot.score > 0.0]

,count,mean,nama_MUA,weight,score,desc,distance,sim,sim_final
0,1,5.000000,Laura Make Up & Bridal,0.336265,1.681323,[la tulipe cover foundation - natural (putih) ...,3.74,0.819851,0.172964
1,2,5.000000,Aulia Rias Pengantin,0.324764,1.623820,[la tulipe cover foundation - suntan (sawo mat...,25.28,0.577673,0.021981
2,1,4.000000,Ulfa Beauty MUA,0.347180,1.388719,[la tulipe cover foundation - suntan (sawo mat...,49.89,0.335496,0.006593
3,2,4.000000,Intan Rias Pengantin,0.324764,1.299056,[la tulipe cover foundation - natural (putih) ...,8.74,0.577673,0.059309
4,3,4.000000,Griya Putri Make Up,0.321007,1.284027,[Wardah Everyday Luminous Liquid Foundation - ...,2.64,0.395066,0.108535
5,1,4.000000,Liyan Make Up Jember,0.313492,1.253967,[la tulipe cover foundation - natural (putih) ...,1076.30,0.819851,0.000761
6,5,4.600000,Yulia Salon & Bridal,0.150030,0.690139,[Maybelline Fit Me Matte + Poreless Liquid Fou...,25.56,0.218430,0.008224
7,3,3.666667,Larasnada Make Up,0.111919,0.410369,[la tulipe cover foundation - suntan (sawo mat...,6.53,0.398010,0.052857


## COLLABORATIVE FILTERING

In [160]:
def colab_evaluation(data_for_model, data_for_user, selected_user, query_user):
    mae_col = []
    for u in range(len(selected_user)):
        _, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[u], query_user[u])
        nama_mua, _, part_of_model = value_of_mua(query_data_todict, part_of_model)
        pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        col_model = pivot.columns.tolist()
        val_model = np.array(pivot.values.tolist(), dtype=float)
        nama_user = np.array(pivot.index.tolist())
        pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_query = pivot_query.values.tolist()
        col_query = pivot_query.columns.tolist()
        feature = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_query)):
                if col_model[i] == col_query[j]:
                    feature[i] = val_query[0][j]
                    break
        knn = NearestNeighbors(metric='cosine', algorithm='brute')
        knn.fit(val_model)
        distances, indices = knn.kneighbors([feature], n_neighbors=10)
        list_nama = []
        for ln in indices[0]:
             list_nama.append(nama_user[ln])
        pivot = data_for_model[data_for_model.nama.isin(list_nama)].reset_index()
        tmp_cos = np.empty(len(pivot), dtype=float)
        for i in range(len(pivot.nama)):
            for j in range(len(list_nama)):
                if pivot.nama[i] == list_nama[j]:
                    tmp_cos[i] = distances[0][j]
        pivot['cos'] = tmp_cos
        pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
        pivot_['cos'] = pivot.groupby("nama_MUA").cos.agg(['mean'])
        value_rating = np.zeros(len(col_model), dtype=float)
        for i in range(len(pivot_.index)):
            for j in range(len(col_model)):
                if value_rating[j] == 0.0:
                    if col_model[j] == pivot_.index[i]:
                        value_rating[j] = pivot_["mean"][i] * pivot_["cos"][i]
                else:
                    continue
        tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
        mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
        pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_label = pivot_label.values.tolist()
        col_label = pivot_label.columns.tolist()
        final_label = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_label)):
                if col_model[i] == col_label[j]:
                    final_label[i] = val_label[0][j]
                    break
        mae = mean_absolute_error(value_rating, final_label)
        mae_col.append(mae)
        
    mean_mae_col = np.mean(mae_col)
    return mua_recs, mean_mae_col

In [161]:
colab_based_fil, mae_col = colab_evaluation(data_for_model, data_for_user, selected_user, query_user)

In [162]:
colab_based_fil[colab_based_fil.score > 0.0]

,nama_mua,score
0,Nonarika Make Up & Hairdo,5.000000
1,Gasebo Make Up Dekorasi,5.000000
2,Rias Pengantin Ana,5.000000
3,Rias Ning Lida,4.000000
4,Griya Citra Ayu,4.000000
5,Yayuk Makeover,4.000000
6,Ayuk Rias Pengantin,3.640682
7,Ayunia Bridal dan Salon,3.300852
8,Sanggar Rias Pengantin Dewi Balqis,2.912546
9,Laura Make Up & Bridal,2.735072


In [163]:
mae_col

1.2552779994182244

## CONTENT BASED FILTERING

In [164]:
def content_evaluation(data_for_model, data_for_user, selected_user, query_user):
    mae_con = []
    for u in range(len(selected_user)):
        _, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[u], query_user[u])
        nama_mua, _, part_of_model = value_of_mua(query_data_todict, part_of_model)
        pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        col_model = pivot.columns.tolist()
        get_user = part_of_model.groupby("nama").sim_num.agg(["mean"])
        get_user = get_user.sort_values(by=["mean"], ascending=False)
        list_nama = get_user.index[:10].tolist()
        pivot = part_of_model[part_of_model.nama.isin(list_nama)].reset_index()
        pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
        pivot_['cos'] = pivot.groupby("nama_MUA").sim_num.agg(['mean'])
        value_rating = np.zeros(len(col_model), dtype=float)
        for i in range(len(pivot_.index)):
            for j in range(len(col_model)):
                if value_rating[j] == 0.0:
                    if col_model[j] == pivot_.index[i]:
                        value_rating[j] = pivot_["mean"][i] * pivot_["cos"][i]
                else:
                    continue
        tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
        mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
        pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_label = pivot_label.values.tolist()
        col_label = pivot_label.columns.tolist()
        final_label = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_label)):
                if col_model[i] == col_label[j]:
                    final_label[i] = val_label[0][j]
                    break
        mae = mean_absolute_error(value_rating, final_label)
        mae_con.append(mae)
        
    mean_mae_con = np.mean(mae_con)
    return mua_recs, mean_mae_con

In [165]:
content_based_fil, mae_con = content_evaluation(data_for_model, data_for_user, selected_user, query_user)

In [166]:
content_based_fil[content_based_fil.score > 0.0]

,nama_mua,score
0,Dinda Wijaya Make Up,3.450126
1,Abha Beauty Make Up Artist,3.234082
2,Sanggar Rias Nining,3.192502
3,RR Make Up,3.128324
4,Gasebo Make Up Dekorasi,3.111091
5,Rias Ning Lida,2.561029
6,Arachma Make Up,2.504092
7,Bonita MUA Jember,2.472246
8,Larasnada Make Up,2.465769
9,Sanggar Rias dan Salon Esthie,1.915119


In [167]:
mae_con

0.868907670433759

In [168]:
data_evaluasi = {
    'Metode' : ['Collaborative Filtering', 'Content Based Filtering', 'Hybrid Filtering'],
    'MAE' : [mae_col, mae_con, mae_hyb]
}

tabel_evaluasi = pd.DataFrame(data_evaluasi)
tabel_evaluasi = tabel_evaluasi.sort_values(by=['MAE'], ignore_index=True)
tabel_evaluasi

,Metode,MAE
0,Hybrid Filtering,0.772906
1,Content Based Filtering,0.868908
2,Collaborative Filtering,1.255278
